In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.utils as vutils


In [ ]:
% load_ext autoreload 
% autoreload 2

In [ ]:
from modules_tied import VAE
from modules_tied import NetD
from modules_tied import Dummy
from modules_tied import loss_function

In [ ]:
bsz = 100
criterion = nn.BCELoss()


In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)


In [ ]:
netG = VAE()
netD = NetD()
dummy = Dummy()

In [ ]:
optimizerD = optim.Adam(netD.parameters(), lr=1e-4)
optimizerG = optim.Adam(netG.parameters(), lr=1e-4)
optimizer_dummy = optim.Adam(dummy.parameters(), lr=1e-4)

In [ ]:
input = torch.FloatTensor(bsz,28,28)
label = torch.FloatTensor(bsz)
real_label=1
fake_label=0
USE_CUDA=1

if(USE_CUDA):
    netG=netG.cuda()
    netD=netD.cuda()
    #dummy=dummy.cuda()
    criterion=criterion.cuda()
    input,label=input.cuda(), label.cuda()

In [ ]:
for epoch in range(10000):
    for i, (data,_) in enumerate(train_loader):
        #update D: maximize log((D(x)) + log(1-D(G(z)))
        netD.zero_grad()
        #ummy.zero_grad()
        
        real_cpu = data
        
        if(USE_CUDA):
            real_cpu = real_cpu.cuda()

        input.resize_as_(real_cpu).copy_(real_cpu)
        label.resize_(bsz).fill_(real_label)

        inputv = Variable(input)
        labelv = Variable(label)

        _,output = netD(inputv)
        errD_real = criterion(output, labelv)
        errD_real.backward()
        D_x = output.data.mean()

        #train with fake
        fake,mu,logvar = netG(inputv)
        labelv = Variable(label.fill_(fake_label))
        _,output = netD(fake.detach())
        errD_fake = criterion(output, labelv)
        errD_fake.backward()

       # zd = torch.FloatTensor(bsz,20).normal_(0,1).cuda()
       # zd = Variable(zd)
       # x_dummy = dummy(zd,netG)
       # _,out_dummy = netD(x_dummy.detach())
       # L_dummy = criterion(out_dummy, labelv)
       # L_dummy.backward()
        
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake #+ L_dummy
        optimizerD.step()

        #step dummy
        #x_dummy = dummy(zd,netG)
        #_,out_dummy = netD(x_dummy.detach())
        #labelv = Variable(label.fill_(real_label))
        #LGAN_dummy = criterion(out_dummy,labelv)
        #LGAN_dummy.backward()
        #_,out_orig = netD(inputv)
        #L_recon_dummy = loss_function(out_dummy.detach(),out_orig,mu.detach(),logvar.detach())
        #L_recon_dummy.backward()
        #optimizer_dummy.step()
        
        
        # update G: maximize log((D(G(z))
        if epoch%1==0:
            netG.zero_grad()
            labelv = Variable(label.fill_(real_label))
            x_tilde, output = netD(fake)
            x_l, output_orig = netD(inputv)
            #errG = criterion(output,labelv) + loss_function(fake,inputv,mu,logvar)
            #errG = loss_function(output,output_orig,mu,logvar) #+ 0.5*criterion(output,labelv)
            #errG = #loss_function(fake,inputv,mu,logvar) +loss_function(output,output_orig,mu,logvar)
            errG = loss_function(x_tilde,x_l,mu,logvar)
            #errG = loss_function(fake,inputv,mu,logvar)

            errG.backward()
    
            D_G_z2 = output.data.mean()
            optimizerG.step()

        if(epoch %1 ==0):
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
                  % (epoch, 100, i, len(train_loader),
                     errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2))
            if i % 100 == 0:
                print('real_cpu.size()', real_cpu.size())
                vutils.save_image(real_cpu,
                                  './real_samples.png',
                                  normalize=True)
                vutils.save_image(fake.data.view(-1,1,28,28),
                                  './fake_samples.png',
                                  normalize=True)
